In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch_geometric.nn import MessagePassing, DenseGCNConv, DenseGraphConv, global_mean_pool, ASAPooling, GATv2Conv, GCNConv, dense_diff_pool
from torch_geometric.utils import add_self_loops, degree, scatter
import numpy as np
import torch
import os.path as osp
import torch
from torch.nn import Linear
import torch_scatter
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GraphConv, dense_mincut_pool
from torch_geometric import utils
from torch_geometric.nn import Sequential
from torch_geometric.nn.conv.gcn_conv import gcn_norm

OSError: [WinError 127] The specified procedure could not be found

In [ ]:
dl = torch.load( '../../data/MLgSA/export/train_loader.pt')

In [3]:
class VarianceAwareMessagePassing(MessagePassing):
    def __init__(self, in_channels, out_channels, epsilon=1e-5):
        super(VarianceAwareMessagePassing, self).__init__(aggr='mean')  # Use mean aggregation
        self.epsilon = epsilon
        self.lin = nn.Linear(in_channels, out_channels)
        self.update_mlp = nn.Sequential(
            nn.Linear(out_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )

    def forward(self, x, edge_index):
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        
        # Step 1: Linear transformation
        x = self.lin(x)

        # Step 2: Propagate messages
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j, x_i):
        # x_j: Source node features
        # x_i: Target node features
        # Compute message as difference of features
        return x_j - x_i
    
    def aggregate(self, inputs, index):
        # inputs: Messages from neighbors [num_edges, out_channels]
        # index: Target nodes for each edge [num_edges]

        # Compute the mean of the messages for each target node
        mean_messages = scatter_mean(inputs, index, dim=0)
        mean_messages_expanded = mean_messages[index]

        # Compute the variance of the messages for each target node
        sq_diff = (inputs - mean_messages_expanded) ** 2
        mean_sq_diff = scatter_mean(sq_diff, index, dim=0)
        
        # Compute importance weight based on variance
        importance_weight = torch.where(mean_sq_diff > self.epsilon, mean_sq_diff, torch.ones_like(mean_sq_diff))
        importance_weight_expanded = importance_weight[index]

        # Weight the inputs by the computed importance weight
        weighted_inputs = inputs * importance_weight_expanded.unsqueeze(-1)
        return super().aggregate(weighted_inputs, index)
    
    def update(self, aggr_out, x):
        # aggr_out: Aggregated messages
        # x: Initial node features
        return self.update_mlp(aggr_out) + x

# Example usage
import torch_geometric
from torch_geometric.datasets import Planetoid

# Load dataset (e.g., Cora)
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = VarianceAwareMessagePassing(in_channels, hidden_channels)
        self.conv2 = VarianceAwareMessagePassing(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel(dataset.num_node_features, 16, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
accuracy = correct / data.test_mask.sum().item()
print('Test Accuracy: {:.4f}'.format(accuracy))

TypeError: scatter() got an unexpected keyword argument 'return_counts'